In [1]:
# The goal of this project is to create a model from the fifa data which is given:
#     - The first step is to clean the data 

In [2]:
# importing all the required libraries
import pandas as pd
import numpy as np
import math
from sklearn import linear_model

import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import pickle 

In [3]:
# importing the data file
df= pd.read_csv("fifa21_train.csv")
df

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,...,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2,65
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,"6'1""",174lbs,...,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2,70
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,"5'6""",143lbs,...,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2,67
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,...,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68


# Data exploring and cleaning

In [4]:
# finding the information of the df to know the dtypes 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11701 entries, 0 to 11700
Columns: 101 entries, ID to OVA
dtypes: float64(10), int64(43), object(48)
memory usage: 9.0+ MB


In [5]:
# finding the column names
df.columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Club', 'BP', 'Position',
       'Team & Contract', 'Height', 'Weight',
       ...
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK', 'OVA'],
      dtype='object', length=101)

In [6]:
# finding the total count of null values in df
df.isna().sum()

ID              0
Name            0
Age             0
Nationality     0
Club           18
               ..
CB              0
RCB             0
RB              0
GK              0
OVA             0
Length: 101, dtype: int64

In [7]:
# to see the maximum number of columns 
pd.set_option('display.max_columns', None)

# to see the maximum number of columns 
# pd.set_option('display.max_rows', None)

In [8]:
# finding all the columns in the list format
columns = list(df.columns)
print(columns)

['ID', 'Name', 'Age', 'Nationality', 'Club', 'BP', 'Position', 'Team & Contract', 'Height', 'Weight', 'foot', 'Growth', 'Joined', 'Loan Date End', 'Value', 'Wage', 'Release Clause', 'Contract', 'Attacking', 'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats', 'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Hits', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB'

In [9]:
# finding shape to know how many rows and columns are present
df.shape

(11701, 101)

In [10]:
# Change headers names to lower case
df.columns= [col.lower() for col in df.columns]
df.rename(columns={'st':'state'},inplace=True)
df.head()

,id,name,age,nationality,club,bp,position,team & contract,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [11]:
# The beolw code is used to replace space with underscore in the header: best position--> best_position
# def clean_data(df):
#     df.columns=[e.lower().replace(' ', '_') for e in df.columns]
#     df=df.drop(columns=['unnamed:_0', 'vehicle_type'])
#     df=df.dropna()
#     df['effective_to_date']=pd.to_datetime(df['effective_to_date'], errors='coerce')
#     return df

In [12]:
#Categorical Features- extracting the columns which are object dtypes
df_c=df.select_dtypes(include = np.object)
df_c.tail()

,name,nationality,club,bp,position,team & contract,height,weight,foot,joined,loan date end,value,wage,release clause,contract,w/f,sm,a/w,d/w,ir,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
11696,B. Böðvarsson,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,Left,"Feb 8, 2018",NaN,€625K,€3K,€891K,2018 ~ 2021,3 ★,2★,Medium,Medium,1 ★,3,48+2,48+2,48+2,55+0,52+0,52+0,52+0,55+0,54+2,54+2,54+2,58+2,56+2,56+2,56+2,58+2,63+2,60+2,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2
11697,G. Gallon,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,"6'1""",174lbs,Right,"Jul 1, 2019",NaN,€1.6M,€4K,€2.9M,2019 ~ 2022,2 ★,1★,Medium,Medium,1 ★,3,28+2,28+2,28+2,24+0,26+0,26+0,26+0,24+0,24+2,24+2,24+2,25+2,25+2,25+2,25+2,25+2,25+2,26+2,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2
11698,J. Flores,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,"5'6""",143lbs,Right,"Jan 23, 2019",NaN,€1.1M,€2K,€1.7M,2019 ~ 2024,3 ★,3★,High,Medium,1 ★,6,62+2,62+2,62+2,67+0,65+0,65+0,65+0,67+0,65+2,65+2,65+2,66+2,58+2,58+2,58+2,66+2,49+2,44+2,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2
11699,Anderson Silva,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,Right,NaN,NaN,€0,€0,€0,2010,3 ★,1★,NaN,NaN,3 ★,3,68+0,68+0,68+0,71+0,68+0,68+0,68+0,71+0,72+0,72+0,72+0,71+0,71+0,71+0,71+0,71+0,66+0,68+0,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0
11700,T. Conechny,Argentina,Portland Timbers,CAM,ST CAM LM,Portland Timbers 2019 ~ 2020,"5'7""",143lbs,Left,"Jul 22, 2019",NaN,€700K,€2K,€1.2M,2019 ~ 2020,3 ★,3★,High,Low,1 ★,8,64+2,64+2,64+2,65+0,65+0,65+0,65+0,65+0,65+2,65+2,65+2,63+2,58+2,58+2,58+2,63+2,46+2,43+2,43+2,43+2,46+2,44+2,36+2,36+2,36+2,44+2,15+2


In [13]:
#numerical Features- extracting the columns which are numerical dtypes
df_n=df.select_dtypes(include = np.number)
df_n.head()

,id,age,growth,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,ova
0,184383,26,1,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,69,51,63,63,51,60,64
1,188044,30,0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,83,75,68,82,33,71,77
2,184431,33,0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,80,77,78,86,27,56,80
3,233796,22,13,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,57,44,54,57,57,60,59
4,234799,23,8,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,66,44,60,64,60,66,65


In [14]:
# finding null values through out numerical df
df_n.isna().sum()

id                    0
age                   0
growth                0
attacking             0
crossing              0
finishing             0
heading accuracy      0
short passing         0
volleys              41
skill                 0
dribbling             0
curve                41
fk accuracy           0
long passing          0
ball control          0
movement              0
acceleration          0
sprint speed          0
agility              41
reactions             0
balance              41
power                 0
shot power            0
jumping              41
stamina               0
strength              0
long shots            0
mentality             0
aggression            0
interceptions         7
positioning           7
vision               41
penalties             0
composure           279
defending             0
marking               0
standing tackle       0
sliding tackle       41
goalkeeping           0
gk diving             0
gk handling           0
gk kicking      

In [15]:
# def show_categorical_values(df):
#     # Get the list of categorical columns
#     categorical_columns = df.select_dtypes(include='object').columns

#     # Print unique values for each categorical column
#     for column in categorical_columns:
#         print(f"Unique values for column '{column}':")
#         print(df[column].unique())
#         print("\n")

In [16]:
# finding the unique values using .value_counts()
#- here it looks this column has lot of unique values 
# it is not preferred for encoding when there are many unique values                                                 
df_c["nationality"].value_counts()

England                     1162
Germany                      778
Spain                        744
France                       675
Brazil                       567
                            ... 
Tanzania                       1
Jordan                         1
Puerto Rico                    1
Belize                         1
Central African Republic       1
Name: nationality, Length: 159, dtype: int64

In [17]:
# to know the categorical data 
df_c.head()

,name,nationality,club,bp,position,team & contract,height,weight,foot,joined,loan date end,value,wage,release clause,contract,w/f,sm,a/w,d/w,ir,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,4 ★,2★,High,Medium,1 ★,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1
1,Alan Carvalho,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",3 ★,4★,High,Low,2 ★,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2
2,S. Giovinco,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,4 ★,4★,High,Medium,2 ★,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2
3,J. Evans,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,2 ★,2★,Medium,Medium,1 ★,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2
4,Y. Demoncy,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,2 ★,3★,Low,Medium,1 ★,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2


In [18]:
# finding the sum of duplicates in the df which is not much important according to this data or the output shown
duplicates_count = df.apply(lambda column: column.duplicated().sum())
# duplicates_count["name"]
duplicates_count

id                 0
name             469
age            11673
nationality    11542
club           10815
               ...  
cb             11388
rcb            11388
rb             11439
gk             11546
ova            11651
Length: 101, dtype: int64

In [19]:
# finding the total nan values in each column from the df
nan_count = df.isnull().sum()

# Create a new DataFrame with NaN count
nan_count_df = pd.DataFrame({'Column': nan_count.index, 'NaN Count': nan_count.values})

# print("NaN count in each column:")
nan_count_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
Column,id,name,age,nationality,club,bp,position,team & contract,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
NaN Count,0,0,0,0,18,0,288,0,0,0,0,0,41,11072,0,0,0,0,0,0,0,0,0,41,0,0,41,0,0,0,0,0,0,41,0,41,0,0,41,0,0,0,0,0,7,7,41,0,279,0,0,0,41,0,0,0,0,0,0,0,0,0,0,59,59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
# along with nan values finding the percentage of nan values in df
# this step is not important for this task

# Calculate percentage of NaN values in each column
total_rows = len(df)
nan_percentage = (nan_count / total_rows) * 100

# Create a new DataFrame to display the counts and percentages
nan_summary = pd.DataFrame({
    'NaN Count': nan_count,
    'NaN Percentage': nan_percentage
})

print("NaN Count and Percentage in each column:")
nan_summary.T

NaN Count and Percentage in each column:


,id,name,age,nationality,club,bp,position,team & contract,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
NaN Count,0.0,0.0,0.0,0.0,18.000000,0.0,288.000000,0.0,0.0,0.0,0.0,0.0,41.000000,11072.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.000000,0.0,0.0,41.000000,0.0,0.0,0.0,0.0,0.0,0.0,41.000000,0.0,41.000000,0.0,0.0,41.000000,0.0,0.0,0.0,0.0,0.0,7.000000,7.000000,41.000000,0.0,279.000000,0.0,0.0,0.0,41.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.00000,59.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NaN Percentage,0.0,0.0,0.0,0.0,0.153833,0.0,2.461328,0.0,0.0,0.0,0.0,0.0,0.350397,94.624391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.350397,0.0,0.0,0.350397,0.0,0.0,0.0,0.0,0.0,0.0,0.350397,0.0,0.350397,0.0,0.0,0.350397,0.0,0.0,0.0,0.0,0.0,0.059824,0.059824,0.350397,0.0,2.384412,0.0,0.0,0.0,0.350397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50423,0.50423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data transformation

In [21]:
# # converting height into cm from feet and inches- because height is numeric it should be in numeric column
# def convert_to_cm(height):
#     feet, inches = map(int, height.replace('"', '').split("'"))
#     total_inches = (feet * 12) + inches
#     return total_inches * 2.54

# # Convert height values to centimeters
# df['height_cm'] = df['height'].apply(convert_to_cm)

# # Drop the original height column if needed
# df.drop(['height'], axis=1, inplace=True)

# # print("DataFrame after converting height to centimeters:")
# df.head()

In [22]:
# df['height_cm'].dtype

In [23]:
# # removing lbs from weight column
# df['weight_lbs'] = df['weight'].str.replace('lbs', '')
# df=df.drop(['weight'], axis=1)
# df.head()

In [24]:
# df['weight_lbs'] = pd.to_numeric(df['weight_lbs'], errors='coerce', downcast='integer')


In [25]:
# df['weight_lbs'].dtype

In [26]:
# df.head()

In [27]:
# df['joined'] = pd.to_datetime(df['joined'])
# df['loan date end'] = pd.to_datetime(df['loan date end'])

# # # Convert the datetime to "01-01-2015" format
# # df['Formatted_joined'] = df['joined'].dt.strftime('%m-%d-%Y')
# df.head()

In [28]:
def convert_currency(currency_str):
    if currency_str.endswith('K'):
        return float(currency_str[1:-1]) * 1000
    elif currency_str.endswith('M'):
        return float(currency_str[1:-1]) * 1000000
    else:
        return float(currency_str[1:])
# Apply the conversion function to the 'Salary' column
df['value'] = df['value'].apply(convert_currency)
df['wage'] = df['wage'].apply(convert_currency)
df['release clause'] = df['release clause'].apply(convert_currency)


df.tail()

,id,name,age,nationality,club,bp,position,team & contract,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,Left,5,"Feb 8, 2018",NaN,625000.0,3000.0,891000.0,2018 ~ 2021,224,62,24,51,59,28.0,248,57,49.0,29,54,59,333,78,73,68.0,53,61.0,273,31,72.0,77,68,25,266,63,59.0,51.0,54.0,39,46.0,184,59,64,61.0,61,13,14,14,6,14,1589,347,3 ★,2★,Medium,Medium,1 ★,75,28,56,59,60,69,3,48+2,48+2,48+2,55+0,52+0,52+0,52+0,55+0,54+2,54+2,54+2,58+2,56+2,56+2,56+2,58+2,63+2,60+2,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2,65
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,"6'1""",174lbs,Right,4,"Jul 1, 2019",NaN,1600000.0,4000.0,2900000.0,2019 ~ 2022,84,12,14,13,29,16.0,96,17,15.0,19,27,18,237,40,40,38.0,65,54.0,217,50,59.0,29,64,15,101,30,22.0,15.0,15.0,19,56.0,41,12,16,13.0,347,72,67,66,70,72,1123,387,2 ★,1★,Medium,Medium,1 ★,72,67,66,72,40,70,3,28+2,28+2,28+2,24+0,26+0,26+0,26+0,24+0,24+2,24+2,24+2,25+2,25+2,25+2,25+2,25+2,25+2,26+2,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2,70
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,"5'6""",143lbs,Right,8,"Jan 23, 2019",NaN,1100000.0,2000.0,1700000.0,2019 ~ 2024,286,64,66,51,64,41.0,291,71,57.0,38,58,67,388,77,76,86.0,62,87.0,278,61,71.0,64,37,45,241,27,25.0,66.0,58.0,65,58.0,62,37,12,13.0,57,13,10,11,7,16,1603,337,3 ★,3★,High,Medium,1 ★,76,60,60,72,26,43,6,62+2,62+2,62+2,67+0,65+0,65+0,65+0,67+0,65+2,65+2,65+2,66+2,58+2,58+2,58+2,66+2,49+2,44+2,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2,67
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,Right,7,NaN,NaN,0.0,0.0,0.0,2010,254,64,66,51,73,NaN,284,74,NaN,63,72,75,198,71,72,NaN,55,NaN,258,61,NaN,71,64,62,241,64,61.0,63.0,NaN,53,NaN,113,55,58,NaN,81,9,24,72,24,24,1429,400,3 ★,1★,NaN,NaN,3 ★,72,63,70,72,57,66,3,68+0,68+0,68+0,71+0,68+0,68+0,68+0,71+0,72+0,72+0,72+0,71+0,71+0,71+0,71+0,71+0,66+0,68+0,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68
11700,236507,T. Conechny,22,Argentina,Portland Timbers,CAM,ST CAM LM,Portland Timbers 2019 ~ 2020,"5'7""",143lbs,Left,9,"Jul 22, 2019",NaN,700000.0,2000.0,1200000.0,2019 ~ 2020,289,43,64,65,64,53.0,267,64,41.0,43,55,64,391,81,78,87.0,61,84.0,312,62,85.0,65,42,58,244,34,20.0,66.0,63.0,61,56.0,56,20,18,18.0,48,10,8,14,8,8,1607,336,3 ★,3★,High,Low,1 ★,79,62,56,67,24,48,8,64+2,64+2,64+2,65+0,65+0,65+0,65+0,65+0,65+2,65+2,65+2,63+2,58+2,58+2,58+2,63+2,46+2,43+2,43+2,43+2,46+2,44+2,36+2,36+2,36+2,44+2,15+2,64


In [29]:
# df["joined"].dtype

In [30]:
# df["loan date end"].dtype

In [31]:
# df['joined'] = df['joined'].astype('object')
# df['loan date end'] = df['loan date end'].astype('object')


In [32]:
# df["joined"].dtype

In [33]:
# df["loan date end"].dtype

In [34]:
# drop team & contract,value, wage andrelease clause from df
df=df.drop(['team & contract'], axis=1)

df

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,525000.0,4000.0,801000.0,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",8500000.0,23000.0,0.0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,9000000.0,49000.0,15300000.0,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,275000.0,4000.0,694000.0,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,725000.0,2000.0,1400000.0,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,"6'1""",168lbs,Left,5,"Feb 8, 2018",NaN,625000.0,3000.0,891000.0,2018 ~ 2021,224,62,24,51,59,28.0,248,57,49.0,29,54,59,333,78,73,68.0,53,61.0,273,31,72.0,77,68,25,266,63,59.0,51.0,54.0,39,46.0,184,59,64,61.0,61,13,14,14,6,14,1589,347,3 ★,2★,Medium,Medium,1 ★,75,28,56,59,60,69,3,48+2,48+2,48+2,55+0,52+0,52+0,52+0,55+0,54+2,54+2,54+2,58+2,56+2,56+2,56+2,58+2,63+2,60+2,60+2,60+2,63+2,63+2,61+2,61+2,61+

In [35]:
# converting object into int datatype
df['hits'] = pd.to_numeric(df['hits'], errors='coerce', downcast='integer')
df.head()

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,525000.0,4000.0,801000.0,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3.0,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",8500000.0,23000.0,0.0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44.0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,9000000.0,49000.0,15300000.0,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73.0,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,275000.0,4000.0,694000.0,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7.0,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,725000.0,2000.0,1400000.0,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4.0,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [36]:

# Split the column and remove the '+' with the values after them
columns = ['ls', 'state', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm',
           'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb','gk']
# df[columns]

for colmun in columns: # for itertion of columns
    df[colmun] = df[colmun].str.split('+',expand=True)[0] # Split of string
    df[colmun] = df[colmun].str.split('-',expand=True)[0]
df.head(1)

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,525000.0,4000.0,801000.0,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3.0,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,64


In [37]:
# converting all the +2,+3 columns into numeric
for column in columns:
    df[column] = pd.to_numeric(df[column], errors='coerce', downcast='float')
df.head(1)

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,525000.0,4000.0,801000.0,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3.0,58.0,58.0,58.0,61.0,62.0,62.0,62.0,61.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,59.0,59.0,59.0,59.0,59.0,58.0,54.0,54.0,54.0,58.0,15.0,64


In [38]:
df_c=df.select_dtypes(include = np.object)
df_c.head(1)

,name,nationality,club,bp,position,height,weight,foot,joined,loan date end,contract,w/f,sm,a/w,d/w,ir
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161lbs,Right,"Jul 1, 2015",NaN,2015 ~ 2020,4 ★,2★,High,Medium,1 ★


In [39]:
# finding mode and filling mode value with nan values
column_modes = df_c.mode().iloc[0]

df_c = df_c.fillna(column_modes)
df_c.head(1)

,name,nationality,club,bp,position,height,weight,foot,joined,loan date end,contract,w/f,sm,a/w,d/w,ir
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161lbs,Right,"Jul 1, 2015","Jun 30, 2021",2015 ~ 2020,4 ★,2★,High,Medium,1 ★


In [40]:
# columns_to_delete = ['name', 'nationality','club','bp','position','joined','loan date end','contract']
columns_to_delete = ['name','nationality','club','position','height', 'weight','joined','loan date end','contract']

df_c.drop(columns=columns_to_delete, inplace=True)

df_c

,bp,foot,w/f,sm,a/w,d/w,ir
0,CM,Right,4 ★,2★,High,Medium,1 ★
1,ST,Right,3 ★,4★,High,Low,2 ★
2,CAM,Right,4 ★,4★,High,Medium,2 ★
3,CDM,Right,2 ★,2★,Medium,Medium,1 ★
4,CDM,Right,2 ★,3★,Low,Medium,1 ★
...,...,...,...,...,...,...,...
11696,LB,Left,3 ★,2★,Medium,Medium,1 ★
11697,GK,Right,2 ★,1★,Medium,Medium,1 ★
11698,RM,Right,3 ★,3★,High,Medium,1 ★
11699,CM,Right,3 ★,1★,Medium,Medium,3 ★


In [41]:
df_c["bp"].value_counts()

CB     2170
ST     1858
CAM    1535
GK     1081
RM      971
CDM     835
CM      658
LB      633
RB      630
LM      559
RW      228
LWB     172
RWB     166
LW      148
CF       57
Name: bp, dtype: int64

In [42]:
df_c["ir"].value_counts()

1 ★    10359
2 ★     1007
3 ★      285
4 ★       43
5 ★        7
Name: ir, dtype: int64

In [43]:
df_c["sm"].value_counts()

2★    5127
3★    4537
1★    1120
4★     870
5★      47
Name: sm, dtype: int64

In [44]:
df_c["w/f"].value_counts()

3 ★    7219
2 ★    2312
4 ★    1884
5 ★     200
1 ★      86
Name: w/f, dtype: int64

In [45]:
df_c["ir"].value_counts()

1 ★    10359
2 ★     1007
3 ★      285
4 ★       43
5 ★        7
Name: ir, dtype: int64

In [46]:
df_c.isna().sum()

bp      0
foot    0
w/f     0
sm      0
a/w     0
d/w     0
ir      0
dtype: int64

In [47]:
df_n=df.select_dtypes(include = np.number)

df_n.head()

,id,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,26,1,525000.0,4000.0,801000.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,69,51,63,63,51,60,3.0,58.0,58.0,58.0,61.0,62.0,62.0,62.0,61.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,59.0,59.0,59.0,59.0,59.0,58.0,54.0,54.0,54.0,58.0,15.0,64
1,188044,30,0,8500000.0,23000.0,0.0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,83,75,68,82,33,71,44.0,77.0,77.0,77.0,77.0,77.0,77.0,77.0,77.0,76.0,76.0,76.0,76.0,68.0,68.0,68.0,76.0,57.0,53.0,53.0,53.0,57.0,53.0,48.0,48.0,48.0,53.0,18.0,77
2,184431,33,0,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,80,77,78,86,27,56,73.0,73.0,73.0,73.0,80.0,79.0,79.0,79.0,80.0,80.0,80.0,80.0,79.0,74.0,74.0,74.0,79.0,59.0,56.0,56.0,56.0,59.0,53.0,41.0,41.0,41.0,53.0,12.0,80
3,233796,22,13,275000.0,4000.0,694000.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,57,44,54,57,57,60,7.0,50.0,50.0,50.0,51.0,51.0,51.0,51.0,51.0,53.0,53.0,53.0,53.0,56.0,56.0,56.0,53.0,56.0,58.0,58.0,58.0,56.0,57.0,58.0,58.0,58.0,57.0,14.0,59
4,234799,23,8,725000.0,2000.0,1400000.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,66,44,60,64,60,66,4.0,56.0,56.0,56.0,59.0,59.0,59.0,59.0,59.0,61.0,61.0,61.0,62.0,63.0,63.0,63.0,62.0,64.0,64.0,64.0,64.0,64.0,63.0,61.0,61.0,61.0,63.0,15.0,65


In [48]:
df_n=df_n.drop(['id'], axis=1)
df_n.head()

,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,26,1,525000.0,4000.0,801000.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,69,51,63,63,51,60,3.0,58.0,58.0,58.0,61.0,62.0,62.0,62.0,61.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,59.0,59.0,59.0,59.0,59.0,58.0,54.0,54.0,54.0,58.0,15.0,64
1,30,0,8500000.0,23000.0,0.0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,83,75,68,82,33,71,44.0,77.0,77.0,77.0,77.0,77.0,77.0,77.0,77.0,76.0,76.0,76.0,76.0,68.0,68.0,68.0,76.0,57.0,53.0,53.0,53.0,57.0,53.0,48.0,48.0,48.0,53.0,18.0,77
2,33,0,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,80,77,78,86,27,56,73.0,73.0,73.0,73.0,80.0,79.0,79.0,79.0,80.0,80.0,80.0,80.0,79.0,74.0,74.0,74.0,79.0,59.0,56.0,56.0,56.0,59.0,53.0,41.0,41.0,41.0,53.0,12.0,80
3,22,13,275000.0,4000.0,694000.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,57,44,54,57,57,60,7.0,50.0,50.0,50.0,51.0,51.0,51.0,51.0,51.0,53.0,53.0,53.0,53.0,56.0,56.0,56.0,53.0,56.0,58.0,58.0,58.0,56.0,57.0,58.0,58.0,58.0,57.0,14.0,59
4,23,8,725000.0,2000.0,1400000.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,66,44,60,64,60,66,4.0,56.0,56.0,56.0,59.0,59.0,59.0,59.0,59.0,61.0,61.0,61.0,62.0,63.0,63.0,63.0,62.0,64.0,64.0,64.0,64.0,64.0,63.0,61.0,61.0,61.0,63.0,15.0,65


In [49]:
column_means = df_n.mean()

# Fill NaN values with column means
df_n.fillna(column_means, inplace=True)
df_n

,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,26,1,525000.0,4000.0,801000.0,258,54,47,43,70,44.000000,286,61,44.000000,55,63,63,346,64,73,61.000000,66,82.000000,306,62,73.000000,71,55,45,290,54,52.0,62.0,68.000000,54,54.000000,148,49,56,43.000000,48,7,12,14,9,6,1682,357,69,51,63,63,51,60,3.0,58.0,58.0,58.0,61.0,62.0,62.0,62.0,61.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,59.0,59.0,59.0,59.0,59.0,58.0,54.0,54.0,54.0,58.0,15.0,64
1,30,0,8500000.0,23000.0,0.0,365,66,79,76,68,76.000000,375,83,78.000000,72,63,79,404,83,83,88.000000,75,75.000000,372,74,81.000000,75,74,68,313,54,33.0,78.0,72.000000,76,70.000000,77,35,20,22.000000,55,11,7,14,7,16,1961,412,83,75,68,82,33,71,44.0,77.0,77.0,77.0,77.0,77.0,77.0,77.0,77.0,76.0,76.0,76.0,76.0,68.0,68.0,68.0,76.0,57.0,53.0,53.0,53.0,57.0,53.0,48.0,48.0,48.0,53.0,18.0,77
2,33,0,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.000000,424,85,89.000000,91,74,85,424,84,76,93.000000,78,93.000000,308,79,34.000000,75,42,78,332,75,26.0,80.0,78.000000,73,82.000000,80,23,29,28.000000,21,6,3,6,3,3,1925,404,80,77,78,86,27,56,73.0,73.0,73.0,73.0,80.0,79.0,79.0,79.0,80.0,80.0,80.0,80.0,79.0,74.0,74.0,74.0,79.0,59.0,56.0,56.0,56.0,59.0,53.0,41.0,41.0,41.0,53.0,12.0,80
3,22,13,275000.0,4000.0,694000.0,242,44,42,58,62,36.000000,259,54,41.000000,46,57,61,282,54,59,59.000000,55,55.000000,277,57,60.000000,64,58,38,257,61,57.0,31.0,54.000000,54,48.000000,168,55,58,55.000000,42,8,9,6,7,12,1527,329,57,44,54,57,57,60,7.0,50.0,50.0,50.0,51.0,51.0,51.0,51.0,51.0,53.0,53.0,53.0,53.0,56.0,56.0,56.0,53.0,56.0,58.0,58.0,58.0,56.0,57.0,58.0,58.0,58.0,57.0,14.0,59
4,23,8,725000.0,2000.0,1400000.0,249,49,37,61,68,34.000000,280,64,44.000000,45,61,66,324,66,66,61.000000,62,69.000000,280,61,34.000000,81,61,43,294,66,60.0,55.0,64.000000,49,58.000000,185,58,61,66.000000,52,8,9,15,5,15,1664,360,66,44,60,64,60,66,4.0,56.0,56.0,56.0,59.0,59.0,59.0,59.0,59.0,61.0,61.0,61.0,62.0,63.0,63.0,63.0,62.0,64.0,64.0,64.0,64.0,64.0,63.0,61.0,61.0,61.0,63.0,15.0,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,25,5,625000.0,3000.0,891000.0,224,62,24,51,59,28.000000,248,57,49.000000,29,54,59,333,78,73,68.000000,53,61.000000,273,31,72.000000,77,68,25,266,63,59.0,51.0,54.000000,39,46.000000,184,59,64,61.000000,61,13,14,14,6,14,1589,347,75,28,56,59,60,69,3.0,48.0,48.0,48.0,55.0,52.0,52.0,52.0,55.0,54.0,54.0,54.0,58.0,56.0,56.0,56.0,58.0,63.0,60.0,60.0,60.0,63.0,63.0,61.0,61.0,61.0,63.0,16.0,65
11697,27,4,1600000.0,4000.0,2900000.0,84,12,14,13,29,16.000000,96,17,15.000000,19,27,18,237,40,40,38.000000,65,54.000000,217,50,59.000000,29,64,15,101,30,22.0,15.0,15.000000,19,56.000000,41,12,16,13.000000,347,72,67,66,70,72,1123,387,72,67,66,72,40,70,3.0,28.0,28.0,28.0,24.0,26.0,26.0,26.0,24.0,24.0,24.0,24.0,25.0,25.0,25.0,25.0,25.0,25.0,26.0,26.0,26.0,25.0,24.0,26.0,26.0,26.0,24.0,69.0,70
11698,22,8,1100000.0,2000.0,1700000.0,286,64,66,51,64,41.000000,291,71,57.000000,38,58,67,388,77,76,86.000000,62,87.000000,278,61,71.000000,64,37,45,241,27,25.0,66.0,58.000000,65,58.000000,62,37,12,13.000000,57,13,10,11,7,16,1603,337,76,60,60,72,26,43,6.0,62.0,62.0,62.0,67.0,65.0,65.0,65.0,67.0,65.0,65.0,65.0,66.

In [50]:
df=pd.concat([df_c,df_n], axis=1)
df

,bp,foot,w/f,sm,a/w,d/w,ir,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,CM,Right,4 ★,2★,High,Medium,1 ★,26,1,525000.0,4000.0,801000.0,258,54,47,43,70,44.000000,286,61,44.000000,55,63,63,346,64,73,61.000000,66,82.000000,306,62,73.000000,71,55,45,290,54,52.0,62.0,68.000000,54,54.000000,148,49,56,43.000000,48,7,12,14,9,6,1682,357,69,51,63,63,51,60,3.0,58.0,58.0,58.0,61.0,62.0,62.0,62.0,61.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,59.0,59.0,59.0,59.0,59.0,58.0,54.0,54.0,54.0,58.0,15.0,64
1,ST,Right,3 ★,4★,High,Low,2 ★,30,0,8500000.0,23000.0,0.0,365,66,79,76,68,76.000000,375,83,78.000000,72,63,79,404,83,83,88.000000,75,75.000000,372,74,81.000000,75,74,68,313,54,33.0,78.0,72.000000,76,70.000000,77,35,20,22.000000,55,11,7,14,7,16,1961,412,83,75,68,82,33,71,44.0,77.0,77.0,77.0,77.0,77.0,77.0,77.0,77.0,76.0,76.0,76.0,76.0,68.0,68.0,68.0,76.0,57.0,53.0,53.0,53.0,57.0,53.0,48.0,48.0,48.0,53.0,18.0,77
2,CAM,Right,4 ★,4★,High,Medium,2 ★,33,0,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.000000,424,85,89.000000,91,74,85,424,84,76,93.000000,78,93.000000,308,79,34.000000,75,42,78,332,75,26.0,80.0,78.000000,73,82.000000,80,23,29,28.000000,21,6,3,6,3,3,1925,404,80,77,78,86,27,56,73.0,73.0,73.0,73.0,80.0,79.0,79.0,79.0,80.0,80.0,80.0,80.0,79.0,74.0,74.0,74.0,79.0,59.0,56.0,56.0,56.0,59.0,53.0,41.0,41.0,41.0,53.0,12.0,80
3,CDM,Right,2 ★,2★,Medium,Medium,1 ★,22,13,275000.0,4000.0,694000.0,242,44,42,58,62,36.000000,259,54,41.000000,46,57,61,282,54,59,59.000000,55,55.000000,277,57,60.000000,64,58,38,257,61,57.0,31.0,54.000000,54,48.000000,168,55,58,55.000000,42,8,9,6,7,12,1527,329,57,44,54,57,57,60,7.0,50.0,50.0,50.0,51.0,51.0,51.0,51.0,51.0,53.0,53.0,53.0,53.0,56.0,56.0,56.0,53.0,56.0,58.0,58.0,58.0,56.0,57.0,58.0,58.0,58.0,57.0,14.0,59
4,CDM,Right,2 ★,3★,Low,Medium,1 ★,23,8,725000.0,2000.0,1400000.0,249,49,37,61,68,34.000000,280,64,44.000000,45,61,66,324,66,66,61.000000,62,69.000000,280,61,34.000000,81,61,43,294,66,60.0,55.0,64.000000,49,58.000000,185,58,61,66.000000,52,8,9,15,5,15,1664,360,66,44,60,64,60,66,4.0,56.0,56.0,56.0,59.0,59.0,59.0,59.0,59.0,61.0,61.0,61.0,62.0,63.0,63.0,63.0,62.0,64.0,64.0,64.0,64.0,64.0,63.0,61.0,61.0,61.0,63.0,15.0,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,LB,Left,3 ★,2★,Medium,Medium,1 ★,25,5,625000.0,3000.0,891000.0,224,62,24,51,59,28.000000,248,57,49.000000,29,54,59,333,78,73,68.000000,53,61.000000,273,31,72.000000,77,68,25,266,63,59.0,51.0,54.000000,39,46.000000,184,59,64,61.000000,61,13,14,14,6,14,1589,347,75,28,56,59,60,69,3.0,48.0,48.0,48.0,55.0,52.0,52.0,52.0,55.0,54.0,54.0,54.0,58.0,56.0,56.0,56.0,58.0,63.0,60.0,60.0,60.0,63.0,63.0,61.0,61.0,61.0,63.0,16.0,65
11697,GK,Right,2 ★,1★,Medium,Medium,1 ★,27,4,1600000.0,4000.0,2900000.0,84,12,14,13,29,16.000000,96,17,15.000000,19,27,18,237,40,40,38.000000,65,54.000000,217,50,59.000000,29,64,15,101,30,22.0,15.0,15.000000,19,56.000000,41,12,16,13.000000,347,72,67,66,70,72,1123,387,72,67,66,72,40,70,3.0,28.0,28.0,28.0,24.0,26.0,26.0,26.0,24.0,24.0,24.0,24.0,25.0,25.0,25.0,25.0,25.0,25.0,26.0,26.0,26.0,25.0,24.0,26.0,26.0,26.0,24.0,69.0,70
11698,RM,Right,3 ★,3★,High,Mediu

In [51]:
df_n.isna().sum()

age               0
growth            0
value             0
wage              0
release clause    0
                 ..
cb                0
rcb               0
rb                0
gk                0
ova               0
Length: 83, dtype: int64

## Normalization

In [52]:
#separate the features from the labels
y = df['ova']
X = df.drop(['ova'], axis=1)

In [53]:
#categorical features and numerical ones are going to be treated differently
X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = object)

In [54]:
X_num.describe().T

,count,mean,std,min,25%,50%,75%,max
age,11701.0,2.527049e+01,4.957640e+00,16.0,21.0,25.0,29.0,43.0
growth,11701.0,5.534655e+00,5.810903e+00,-1.0,0.0,4.0,10.0,26.0
value,11701.0,2.543772e+06,5.267533e+06,0.0,375000.0,800000.0,2400000.0,90000000.0
wage,11701.0,1.043483e+04,2.121602e+04,0.0,1000.0,3000.0,11000.0,560000.0
release clause,11701.0,4.526818e+06,1.012383e+07,0.0,499000.0,1300000.0,3900000.0,166500000.0
...,...,...,...,...,...,...,...,...
lcb,11701.0,5.311777e+01,1.457529e+01,16.0,43.0,55.0,65.0,88.0
cb,11701.0,5.311777e+01,1.457529e+01,16.0,43.0,55.0,65.0,88.0
rcb,11701.0,5.311777e+01,1.457529e+01,16.0,43.0,55.0,65.0,88.0
rb,11701.0,5.427442e+01,1.354208e+01,14.0,47.0,57.0,64.0,83.0


In [55]:
from sklearn.preprocessing import MinMaxScaler
MinMaxtransformer = MinMaxScaler().fit(X_num)
X_normalized = MinMaxtransformer.transform(X_num)
print(type(X_normalized))
X_normalized = pd.DataFrame(X_normalized,columns=X_num.columns)
display(X_normalized.head())
print(type(X_normalized))

<class 'numpy.ndarray'>


,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,0.370370,0.074074,0.005833,0.007143,0.004811,0.556931,0.545455,0.478261,0.431818,0.720930,0.465116,0.569087,0.615385,0.444444,0.561798,0.642857,0.637363,0.663818,0.607143,0.729412,0.573171,0.583333,0.8125,0.556270,0.602410,0.685714,0.694118,0.454545,0.455556,0.654596,0.517241,0.522222,0.638298,0.682353,0.534884,0.500000,0.524,0.516854,0.595238,0.440476,0.084309,0.067416,0.116279,0.139535,0.076923,0.056180,0.604577,0.492366,0.614286,0.447368,0.558824,0.522388,0.506494,0.500000,0.000000,0.565789,0.565789,0.565789,0.602564,0.602564,0.602564,0.602564,0.602564,0.610390,0.610390,0.610390,0.631579,0.643836,0.643836,0.643836,0.631579,0.625000,0.614286,0.614286,0.614286,0.625000,0.637681,0.527778,0.527778,0.527778,0.637681,0.075949
1,0.518519,0.037037,0.094444,0.041071,0.000000,0.821782,0.681818,0.826087,0.806818,0.697674,0.837209,0.777518,0.857143,0.822222,0.752809,0.642857,0.813187,0.829060,0.833333,0.847059,0.902439,0.708333,0.7250,0.768489,0.746988,0.800000,0.741176,0.701299,0.711111,0.718663,0.517241,0.311111,0.808511,0.729412,0.790698,0.690476,0.240,0.359551,0.166667,0.190476,0.100703,0.112360,0.058140,0.139535,0.054945,0.168539,0.781945,0.702290,0.814286,0.763158,0.632353,0.805970,0.272727,0.666667,0.042225,0.815789,0.815789,0.815789,0.807692,0.794872,0.794872,0.794872,0.807692,0.779221,0.779221,0.779221,0.802632,0.712329,0.712329,0.712329,0.802632,0.597222,0.528571,0.528571,0.528571,0.597222,0.565217,0.444444,0.444444,0.444444,0.565217,0.113924
2,0.629630,0.037037,0.100000,0.087500,0.091892,0.750000,0.761364,0.793478,0.329545,0.813953,0.825581,0.892272,0.879121,0.944444,0.966292,0.773810,0.879121,0.886040,0.845238,0.764706,0.963415,0.750000,0.9500,0.562701,0.807229,0.128571,0.741176,0.285714,0.822222,0.771588,0.758621,0.233333,0.829787,0.800000,0.755814,0.833333,0.252,0.224719,0.273810,0.261905,0.021077,0.056180,0.011628,0.046512,0.010989,0.022472,0.759059,0.671756,0.771429,0.789474,0.779412,0.865672,0.194805,0.439394,0.072091,0.763158,0.763158,0.763158,0.846154,0.820513,0.820513,0.820513,0.846154,0.831169,0.831169,0.831169,0.842105,0.794521,0.794521,0.794521,0.842105,0.625000,0.571429,0.571429,0.571429,0.625000,0.565217,0.347222,0.347222,0.347222,0.565217,0.037975
3,0.222222,0.518519,0.003056,0.007143,0.004168,0.517327,0.431818,0.423913,0.602273,0.627907,0.372093,0.505855,0.538462,0.411111,0.460674,0.571429,0.615385,0.481481,0.488095,0.564706,0.548780,0.430556,0.4750,0.463023,0.542169,0.500000,0.611765,0.493506,0.377778,0.562674,0.597701,0.577778,0.308511,0.517647,0.534884,0.428571,0.604,0.584270,0.619048,0.583333,0.070258,0.078652,0.081395,0.046512,0.054945,0.123596,0.506039,0.385496,0.442857,0.355263,0.426471,0.432836,0.584416,0.500000,0.004119,0.460526,0.460526,0.460526,0.474359,0.461538,0.461538,0.461538,0.474359,0.480519,0.480519,0.480519,0.500000,0.547945,0.547945,0.547945,0.500000,0.583333,0.600000,0.600000,0.600000,0.583333,0.623188,0.583333,0.583333,0.583333,0.623188,0.063291
4,0.259259,0.333333,0.008056,0.003571,0.008408,0.534653,0.488636,0.369565,0.636364,0.697674,0.348837,0.555035,0.648352,0.444444,0.449438,0.619048,0.670330,0.601140,0.630952,0.647059,0.573171,0.527778,0.6500,0.472669,0.590361,0.128571,0.811765,0.532468,0.433333,0.665738,0.655172,0.611111,0.563830,0.635294,0.476744,0.547619,0.672,0.617978,0.654762,0.714286,0.093677,0.078652,0.081395,0.151163,0.032967,0.157303,0.593134,0.503817,0.571429

<class 'pandas.core.frame.DataFrame'>


In [56]:
pickle.dump(MinMaxtransformer, open('scalar.pkl', 'wb'))

In [57]:
# # plots before normalising
# sns.set(style="whitegrid")
# for column in df_n:
#     plt.figure(figsize=(6, 4))
#     sns.histplot(data=df_n, x=column, kde=True)
#     plt.title(f'Distribution of {column}')
#     plt.show()

In [58]:
# # plots after normalising
# sns.set(style="whitegrid")
# for column in X_normalized:
#     plt.figure(figsize=(6, 4))
#     sns.histplot(data=X_normalized, x=column, kde=True)
#     plt.title(f'Distribution of {column}')
#     plt.show()

# One Hot Encoding (categorical).


In [59]:
X_cat = df.select_dtypes(include = object)

In [60]:
#one hot encoding is a way to turn categorical variables into multiple numerical columns
from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder().fit(X_cat)
encoder = OneHotEncoder(drop='first').fit(X_cat)
print(encoder.categories_)
encoded = encoder.transform(X_cat).toarray()
print(encoded)

[array(['CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LM', 'LW', 'LWB',
       'RB', 'RM', 'RW', 'RWB', 'ST'], dtype=object), array(['Left', 'Right'], dtype=object), array(['1 ★', '2 ★', '3 ★', '4 ★', '5 ★'], dtype=object), array(['1★', '2★', '3★', '4★', '5★'], dtype=object), array(['High', 'Low', 'Medium'], dtype=object), array(['High', 'Low', 'Medium'], dtype=object), array(['1 ★', '2 ★', '3 ★', '4 ★', '5 ★'], dtype=object)]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [61]:
encoder = OneHotEncoder(drop='first').fit(X_cat)

cols = encoder.get_feature_names_out(input_features=X_cat.columns)

X_cat_encode = pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)

X_cat_encode.head()

,bp_CB,bp_CDM,bp_CF,bp_CM,bp_GK,bp_LB,bp_LM,bp_LW,bp_LWB,bp_RB,bp_RM,bp_RW,bp_RWB,bp_ST,foot_Right,w/f_2 ★,w/f_3 ★,w/f_4 ★,w/f_5 ★,sm_2★,sm_3★,sm_4★,sm_5★,a/w_Low,a/w_Medium,d/w_Low,d/w_Medium,ir_2 ★,ir_3 ★,ir_4 ★,ir_5 ★
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [62]:
pickle.dump(encoder, open('encoder.pkl', 'wb'))

In [63]:
# concatinating the normalised and encoded data
X = pd.concat([X_normalized,X_cat_encode], axis=1)  # np.concatenate()
X

,age,growth,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,hits,ls,state,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,bp_CB,bp_CDM,bp_CF,bp_CM,bp_GK,bp_LB,bp_LM,bp_LW,bp_LWB,bp_RB,bp_RM,bp_RW,bp_RWB,bp_ST,foot_Right,w/f_2 ★,w/f_3 ★,w/f_4 ★,w/f_5 ★,sm_2★,sm_3★,sm_4★,sm_5★,a/w_Low,a/w_Medium,d/w_Low,d/w_Medium,ir_2 ★,ir_3 ★,ir_4 ★,ir_5 ★
0,0.370370,0.074074,0.005833,0.007143,0.004811,0.556931,0.545455,0.478261,0.431818,0.720930,0.465116,0.569087,0.615385,0.444444,0.561798,0.642857,0.637363,0.663818,0.607143,0.729412,0.573171,0.583333,0.812500,0.556270,0.602410,0.685714,0.694118,0.454545,0.455556,0.654596,0.517241,0.522222,0.638298,0.682353,0.534884,0.500000,0.524,0.516854,0.595238,0.440476,0.084309,0.067416,0.116279,0.139535,0.076923,0.056180,0.604577,0.492366,0.614286,0.447368,0.558824,0.522388,0.506494,0.500000,0.000000,0.565789,0.565789,0.565789,0.602564,0.602564,0.602564,0.602564,0.602564,0.610390,0.610390,0.610390,0.631579,0.643836,0.643836,0.643836,0.631579,0.625000,0.614286,0.614286,0.614286,0.625000,0.637681,0.527778,0.527778,0.527778,0.637681,0.075949,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.518519,0.037037,0.094444,0.041071,0.000000,0.821782,0.681818,0.826087,0.806818,0.697674,0.837209,0.777518,0.857143,0.822222,0.752809,0.642857,0.813187,0.829060,0.833333,0.847059,0.902439,0.708333,0.725000,0.768489,0.746988,0.800000,0.741176,0.701299,0.711111,0.718663,0.517241,0.311111,0.808511,0.729412,0.790698,0.690476,0.240,0.359551,0.166667,0.190476,0.100703,0.112360,0.058140,0.139535,0.054945,0.168539,0.781945,0.702290,0.814286,0.763158,0.632353,0.805970,0.272727,0.666667,0.042225,0.815789,0.815789,0.815789,0.807692,0.794872,0.794872,0.794872,0.807692,0.779221,0.779221,0.779221,0.802632,0.712329,0.712329,0.712329,0.802632,0.597222,0.528571,0.528571,0.528571,0.597222,0.565217,0.444444,0.444444,0.444444,0.565217,0.113924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.629630,0.037037,0.100000,0.087500,0.091892,0.750000,0.761364,0.793478,0.329545,0.813953,0.825581,0.892272,0.879121,0.944444,0.966292,0.773810,0.879121,0.886040,0.845238,0.764706,0.963415,0.750000,0.950000,0.562701,0.807229,0.128571,0.741176,0.285714,0.822222,0.771588,0.758621,0.233333,0.829787,0.800000,0.755814,0.833333,0.252,0.224719,0.273810,0.261905,0.021077,0.056180,0.011628,0.046512,0.010989,0.022472,0.759059,0.671756,0.771429,0.789474,0.779412,0.865672,0.194805,0.439394,0.072091,0.763158,0.763158,0.763158,0.846154,0.820513,0.820513,0.820513,0.846154,0.831169,0.831169,0.831169,0.842105,0.794521,0.794521,0.794521,0.842105,0.625000,0.571429,0.571429,0.571429,0.625000,0.565217,0.347222,0.347222,0.347222,0.565217,0.037975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.222222,0.518519,0.003056,0.007143,0.004168,0.517327,0.431818,0.423913,0.602273,0.627907,0.372093,0.505855,0.538462,0.411111,0.460674,0.571429,0.615385,0.481481,0.488095,0.564706,0.548780,0.430556,0.475000,0.463023,0.542169,0.500000,0.611765,0.493506,0.377778,0.562674,0.597701,0.577778,0.308511,0.517647,0.534884,0.428571,0.604,0.584270,0.619048,0.583333,0.070258,0.078652,0.081395,0.046512,0.054945,0.123596,0.506039,0.385496,0.442857,0.355263,0.426471,0.432836,0.584416,0.500000,0.004119,0.460526,0.460526,0.460526,0.474359,0.461538,0.461538,0.461538,0.474359,0.480519

In [64]:
# X[""].describe().T

# Splitting into train set and test set.

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9360, 113)
(2341, 113)
(9360,)
(2341,)


# Modeling

In [67]:
# Applying linear regression model
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [68]:
pickle.dump(lm, open('model.pkl', 'wb'))

# Model Validation

In [69]:
from sklearn.metrics import mean_absolute_error, r2_score

In [70]:
lm.predict(X_train)

array([61.16186523, 71.69750977, 67.45092773, ..., 53.5246582 ,
       54.74047852, 71.41186523])

In [71]:
# r2 score
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.9217357278901361

In [72]:
# r2 score
predictions1 = lm.predict(X_test)
r2_score(y_test, predictions1)

0.9172107292516787

In [73]:
from sklearn.metrics import mean_squared_error

# RMSE
rmse = np.sqrt(mean_squared_error(y_test,predictions1))
rmse

1.9799286784992014

In [74]:
#MAE 
mae = mean_absolute_error(y_test, predictions1)
print(mae)

1.5252490004939128


In [75]:
MSE = mean_squared_error(y_train,predictions)
MSE.round(2)

3.7